<a href="https://colab.research.google.com/github/Hubi1001/Game_Player_Analysis_Project/blob/main/BN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Krok 1: Weryfikacja środowiska i instalacja niezbędnych bibliotek

In [87]:
# Instalacja LangChain do budowy łańcucha RAG:
print("Instalacja biblioteki LangChain...")
!pip install -q langchain

# Instalacja bibliotek dla modelu językowego (np. Google Generative AI)
print("\nInstalacja bibliotek dla modelu językowego...")
!pip install -q google-generativeai langchain-google-genai

# Instalacja niezbędnych bibliotek do wektoryzacji i bazy wektorowej:
print("\nInstalacja narzędzi do wektoryzacji i bazy wektorowej...")
!pip install -q pydantic numpy langchain-community sentence-transformers faiss-cpu

Instalacja biblioteki LangChain...

Instalacja bibliotek dla modelu językowego...

Instalacja narzędzi do wektoryzacji i bazy wektorowej...


# Krok 2: Przygotowanie symulowanych danych naukowych / dokumentacji

In [88]:
print("\nŁadowanie i segmentacja symulowanych dokumentów (Analiza Tekstu / Ekstrakcja Informacji):")
documents = [
    "Raport końcowy projektu 123: Zastosowano stop stali 316L. Krytycznym parametrem było napięcie 25 kV na tomografie GE v|tome|x. Pliki VGI zostały skonwertowane do DICOM Multiframe. ID Próbki: A101.",
    "Procedura operacyjna P-04: Metoda analizy dyfrakcyjnej (XRDML) na dyfraktometrze PANalytical Empyrean. Parametry kątowe: 10° do 90°. Wszystkie wyniki muszą być powiązane z metadanymi FAIR.",
    "Notatka laboratoryjna z mikroskopu Leica Stellaris 8 (konfokalny). Obrazy w formacie LIF zostały zindeksowane i są powiązane z próbką P-421. Wykorzystano technikę DICOM Multiframe dla wielokanałowych obrazów.",
    "Opis: Próbka biologiczna, linia komórkowa HeLa. Przechowywana w zamrażarce LIMS F-02 na pozycji A-12. Dane związane z cytometrią (Attune NxT) zostały zarchiwizowane.",
]


Ładowanie i segmentacja symulowanych dokumentów (Analiza Tekstu / Ekstrakcja Informacji):


# Krok 3: Wektoryzacja i stworzenie indeksu (Baza Wektorowa)

In [89]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

# Inicjalizacja modelu osadzeń (Embedding Model)
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Tworzenie indeksu wektorowego z danych
vectorstore = FAISS.from_texts(documents, embeddings)

print("\nIndeks wektorowy FAISS stworzony. System jest gotowy do wyszukiwania semantycznego.")


Indeks wektorowy FAISS stworzony. System jest gotowy do wyszukiwania semantycznego.


# Krok 4: Integracja z Modelem Językowym (LLM) i Utworzenie Łańcucha RAG (Generacja Odpowiedzi)

### Ważne: Dodaj GOOGLE_API_KEY do Colab Secrets!

Aby poniższy kod działał poprawnie, musisz dodać swój klucz API Google Generative AI do Colab Secrets.

1.  Kliknij ikonę **klucza** na lewym pasku bocznym.
2.  Kliknij **"Add new secret"**.
3.  W polu "Name" wpisz **`GOOGLE_API_KEY`**.
4.  W polu "Value" wklej swój klucz API.
5.  Zamknij menedżer sekretów.

In [90]:
# Import niezbędnych klas
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
import google.generativeai as genai
from google.colab import userdata
import os

# Konfiguracja API Key dla Google Generative AI
try:
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key skonfigurowany.")

    # Inicjalizacja modelu językowego
    llm = ChatGoogleGenerativeAI(model="gemini-pro")

    # Utworzenie łańcucha RetrievalQA
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": len(documents)}),
        return_source_documents=True
    )

    print("\nŁańcuch RetrievalQA stworzony. Gotowy do generowania odpowiedzi.")

except userdata.SecretNotFoundError:
    print("Błąd: GOOGLE_API_KEY nie znaleziony w Colab Secrets.")
    print("Proszę dodać GOOGLE_API_KEY do Colab Secrets (ikonka klucza po lewej) i uruchomić ponownie tę komórkę.")
except Exception as e:
    print(f"Wystąpił nieoczekiwany błąd podczas konfiguracji API Key lub inicjalizacji łańcucha RAG: {e}")

API Key skonfigurowany.

Łańcuch RetrievalQA stworzony. Gotowy do generowania odpowiedzi.


# Krok 5: Wykonanie Zapytania z Łańcuchem RAG i Wyświetlenie Odpowiedzi

In [91]:
query = "Jakie napięcie zostało użyte podczas skanowania stali, które dały pliki VGI?"

# Użycie łańcucha RetrievalQA do uzyskania odpowiedzi
print("\nGenerowanie odpowiedzi przy użyciu łańcucha RetrievalQA...")
# Sprawdzenie, czy qa_chain jest zdefiniowane przed użyciem
if 'qa_chain' in locals():
    response = qa_chain.invoke(query)

    # Wyświetlenie pełnej odpowiedzi i źródłowych dokumentów (jeśli zostały zwrócone)
    print("\n--- Pełna Odpowiedź LLM ---")
    print(response['result'])
    print("--------------------------")

    if 'source_documents' in response:
        print("\n--- Źródłowe Dokumenty ---")
        for i, doc in enumerate(response['source_documents']):
            print(f"Dokument {i+1}:")
            print(doc.page_content)
            print("---")
        print("--------------------------")
else:
    print("\nBłąd: qa_chain nie zostało zdefiniowane. Proszę upewnić się, że poprzednia komórka wykonała się pomyślnie po dodaniu klucza API.")


Generowanie odpowiedzi przy użyciu łańcucha RetrievalQA...


Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
    self._credentials.before_request(
  File "/usr/local/lib/python3.12/dist-packages/google/auth/credentials.py", line 239, in before_request
    self._blocking_refresh(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/credentials.py", line 202, in _blocking_refresh
    self.refresh(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 132, in refresh
    raise new_exc from caught_exc
google.auth.exceptions.RefreshError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7b55dfa044a0>)
ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7b55681fb140>" raised exception!
Traceback (most recen

RetryError: Timeout of 600.0s exceeded, last exception: 503 Getting metadata from plugin failed with error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7b55dfe11ca0>)